In [1]:
%load_ext autoreload
%autoreload 2
import pickle
from datetime import datetime
import numpy as np
from qutip import *
import os
from matplotlib import pyplot as plt
import sys
from scipy.integrate import solve_ivp
import sandbox_correct as sb
import importlib
importlib.reload(sb)

<module 'sandbox_correct' from '/home/saha/Desktop/tesis/colera/check_polaron_17_10/sandbox_correct.py'>

In [2]:
### Load the full dynamics from the different already calculated dynamics.

save_dir = "../energy_shift/diff_point1_mid_diss"
filepath = save_dir + "/eff_coupling_6.0MHz.pkl"

with open(filepath, 'rb') as f:
    loaded_data = pickle.load(f)

for keys in loaded_data:
    print(keys)

full_dynamics_normal_shift
full_dynamics_polaron_shift
parameters
timestamp
description


In [3]:
#Get dynamics and parameters.

dynamics = loaded_data['full_dynamics_normal_shift']
parameters = loaded_data['parameters']

N = parameters['N']
cavity_diss_rate = parameters['cavity_diss_rate']
input_power = parameters['input_power']
rabi_freq = parameters['rabi_freq']
eff_coupling = parameters['eff_coupling']
qubit_targetz = parameters['qubit_targetz']
norm = parameters['norm']
final_time__mus = parameters['final_time__mus'] 
time_steps = parameters["time_steps"]
psi0_atom = parameters['initial_state_qubit']
tlist = np.linspace(0, final_time__mus, time_steps)

cavity_photon_number = 10**(input_power/10)
qubit_detuning = qubit_targetz - eff_coupling*(2*cavity_photon_number + 1) 
qubit_detuning_lamb_shift = qubit_detuning + eff_coupling*(2*cavity_photon_number + 1)
cavity_detuning = (qubit_detuning_lamb_shift**2 + rabi_freq**2)**.5
cavity_drive_amplitude = np.real((cavity_photon_number * (cavity_detuning**2 + .25*cavity_diss_rate**2))**.5)
cavity_field = cavity_drive_amplitude/(-cavity_detuning + .5j*cavity_diss_rate)

In [ ]:
# Calculate expectations values.

d = tensor(destroy(N), qeye(2))   
sz = tensor(qeye(N), sigmaz())  
sx = tensor(qeye(N), sigmax())     
sm = tensor(qeye(N), sigmam()) 


x_exp = expect(sx, dynamics)
z_exp = expect(sz, dynamics)
photon_number = expect(d.dag()*d, dynamics)

# Qubit dynamics traced.
qubit_dynamics = np.array([ptrace(i, 1) for i in dynamics])

# Cavity dynamics traced
cavity_dynamics = np.array([ptrace(i, 0) for i in dynamics])
cavity_state_dynamics = np.array([expect(destroy(N), i) for i in cavity_dynamics])

#Calculate ground state expectation values if we cool the qubit to the polaron shifted Hamiltonian.
H_qubit_pol = sb.target_hamiltonian_polaron(parameters)
evalues_target_pol, estates_target_pol = H_qubit_pol.eigenstates()

# For ground state
gs_sigmax_pol = expect(sigmax(), estates_target_pol[0])
gs_sigmaz_pol = expect(sigmaz(), estates_target_pol[0])

# For excited state
es_sigmax_pol = expect(sigmax(), estates_target_pol[1])
es_sigmaz_pol = expect(sigmaz(), estates_target_pol[1])

# Non-polaron cavity state
cavity_initial_state = coherent(N, cavity_field)
cavity_initial_rho = cavity_initial_state @ cavity_initial_state.dag()
photon_number_normal_shift = expect(destroy(N).dag()*destroy(N), cavity_initial_rho)
